In [101]:
import requests
import re
from unidecode import unidecode
from bs4 import BeautifulSoup

In [104]:
class BillFinder:
    
    def __init__(self):
        
        self.source_urls_energy = {'Canada': 'https://en.wikipedia.org/wiki/List_of_Canadian_electric_utilities',
                                   'USA': 'https://en.wikipedia.org/wiki/List_of_United_States_electric_companies',
                                   'UK': 'https://www.utilitysavingexpert.com/energy/suppliers/'}
        
        self.source_urls_water = {'USA': 'https://en.wikipedia.org/wiki/List_of_United_States_water_companies',
                                  'UK': 'https://en.wikipedia.org/wiki/United_Kingdom_water_companies'}
        
        self.source_urls_gas = {'USA': 'https://en.wikipedia.org/wiki/List_of_United_States_natural_gas_companies'}
    
    def get_energy_companies(self, country: str = None):
    
        company_set = set()
        soup = BeautifulSoup(requests.get(self.source_urls_energy[country]).text)
        
        if country == 'Canada':
            for t in soup.find_all('table', role="presentation"):
                for _ in t.find_all('li'):
                    for a in _.find_all('a'):
                        company_set.add(a.text)
        elif country == 'USA':
            for t in soup.find('span', id="List_of_US_electric_companies_by_state"):
                for e in t.parent.next_elements:
                    if e.name == 'table':
                        break
                    if e.name == 'ul':
                        for _ in e.find_all('li'):
                            if company_name := _.text.strip():
                                company_set.add(company_name)
        elif country == 'UK':
            for header_text in ['List of UK energy suppliers', 'List of business energy suppliers']:
                p = soup.find('h2', string=header_text).parent
                for _ in p.find_all('div', class_="table-responsive"):
                    for row in _.find_all('tr'):
                        if tds := row.find_all('td'):
                            company_set.add(tds[1].text)
                            
        print(f'found {len(company_set):,} companies')
                        
        return company_set
    
    def get_water_companies(self, country: str = None):
    
        company_set = set()
        soup = BeautifulSoup(requests.get(self.source_urls_water[country]).text)
        
        if country == 'USA':
            for t in soup.find_all('h2'):
                if t.find_all('a', href=True):
                    if list_of_company_names := t.find_next_sibling('ul'):
                        for name_line in list_of_company_names.find_all('li'):
                            if company_name := name_line.text.strip():
                                company_set.add(company_name)
        elif country == 'UK':
            for s in soup.find('span', id='Water_and_sewerage').parent.next_siblings:
                if s.name == 'table':
                    for row in s.find_all('tr'):
                        if tds := row.find_all('td'):
                            company_set.add(tds[0].text.strip()) 
                    break
            for s in soup.find('span', id='Water_only').parent.next_siblings:
                if s.name == 'table':
                    for row in s.find_all('tr'):
                        if tds := row.find_all('td'):
                            company_set.add(tds[0].text.strip()) 
                    break
                    
            for id_ in ['Scotland', 'Northern_Ireland', 'Crown_dependencies']:
                for s in soup.find('span', id=id_).parent.next_siblings:
                    if s.name == 'ul':
                        for item_ in s.find_all('li'):
                            company_set.add(item_.text)
                        break
                            
        print(f'found {len(company_set):,} companies')
                        
        return company_set
    
    def get_gas_companies(self, country: str = None):
    
        company_set = set()
        soup = BeautifulSoup(requests.get(self.source_urls_gas[country]).text)
        
        if country == 'USA':
            for list_item in soup.find('span', id='List').parent.find_next_sibling('ul').find_all('li'):
                for c in unidecode(list_item.text).split('-')[1].split(','):
                    if company_name := c.strip():
                        company_set.add(company_name)
                        
        print(f'found {len(company_set):,} companies')
        
        return company_set

In [105]:
bf = BillFinder()

In [106]:
bf.get_gas_companies(country='USA')

found 159 companies


{'Aquila',
 'Arkansas Oklahoma Gas Corporation',
 'Atmos Energy',
 'Avista Utilities',
 'Bangor Gas',
 'Berkshire Gas',
 'Black Hills Energy',
 'Black Hills Energy.',
 'Blackstone Gas Company',
 'Bluefield Natural Gas',
 'CNX Resources',
 'Cascade Natural Gas',
 'CenterPoint Energy',
 'CenterPoint Energy Florida Gas Transmission Company',
 'CenterPoint Energy Gas Transmission',
 'Central Hudson Gas & Electric',
 'Chesapeake Energy',
 'Chesapeake Operating Inc',
 'Chesapeake Utilities',
 'Chester County Gas Authority',
 'City Utilities of Springfield',
 'City of Ellensburg',
 'City of Long Beach Gas & Oil Department',
 'City of Norwich Department of Public Utilities',
 'City of Palo Alto Gas Department',
 'CoServ Gas',
 'Colonial Gas',
 'Columbia Gas',
 'Columbia Gas of Maryland',
 'Columbia Gas of Massachusetts',
 'Columbia Gas of Ohio',
 'Columbia Gas of Virginia',
 'ConocoPhillips',
 'Consolidated Edison',
 'Consumers Energy',
 'Cut Bank Gas Company',
 'DTE Energy',
 'Delmarva Power'